# Exploring Tools in LangChain

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.42
    Uninstalling langsmith-0.3.42:
      Successfully uninstalled langsmith-0.3.42
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy

## Install Search APIs

In [2]:
!pip install duckduckgo-search
!pip install wikipedia
# to highlight json
!pip install rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=3aca81009b3f03af29a16b6d1ea41e89f634c5078c6da29ea60c14791ab144fd
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [3]:
import duckduckgo_search
duckduckgo_search.__version__

'8.0.2'

In [4]:
import wikipedia
wikipedia.__version__

(1, 4, 0)

## Get Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

## Setup Environment Variables

In [5]:
from google.colab import userdata
import os

os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Exploring the Wikipedia Tool

Enables you to tap into the Wikipedia API to search wikipedia pages for information

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=8000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper, features="lxml")

In [7]:
wiki_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [8]:
wiki_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [9]:
print(wiki_tool.invoke({"query": "New Delhi"}))

Page: New Delhi
Summary: New Delhi ( ; Hindi: Naī Dillī, pronounced [ˈnəiː ˈdɪlːiː]) is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court. New Delhi is a municipality within the NCT, administered by the New Delhi Municipal Council (NDMC), which covers mostly Lutyens' Delhi and a few adjacent areas. The municipal area is part of a larger administrative district, the New Delhi district.
Although colloquially Delhi and New Delhi are used interchangeably to refer to the National Capital Territory of Delhi, both are distinct entities, with the municipality and the New Delhi district forming a relatively small part within the megacity of Delhi. The National Capital Region is an even larger entity, comprising the entire NCT along with adjoining districts in the two neighbouring states forming a continuously built-up area w

 We can customize the default tool with its own name, description and so on as follows

In [10]:
from langchain.agents import Tool

wiki_tool_init = Tool(name="Wikipedia",
                      func=wiki_api_wrapper.run,
                      description="useful when you need a detailed answer about encyclopaedic general knowledge")

In [11]:
wiki_tool_init.description

'useful when you need a detailed answer about encyclopaedic general knowledge'

In [12]:
wiki_tool_init.args

{'tool_input': {'type': 'string'}}

In [13]:
print(wiki_tool_init.invoke({"tool_input": "AI"}))

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applicat

## Exploring the Tavily Search Tool

Tavily Search API is a search engine optimized for LLMs and RAG, aimed at efficient, quick and persistent search results

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=2)

In [15]:
tavily_tool.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [16]:
tavily_tool.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [17]:
results = tavily_tool.invoke("Tell me about New Delhi")
results

[{'url': 'https://en.wikipedia.org/wiki/New_Delhi',
  'content': "New Delhi (/ˈ nj uː ˈ d ɛ.l i/ⓘ;[6]Hindi: _\\_Naī Dillī\\__, pronounced[ˈnəiː ˈdɪlːiː]) is the capital of India and a part of the National Capital Territory of Delhi (NCT). New Delhi is the seat of all three branches of the Government of India, hosting the Rashtrapati Bhavan, Sansad Bhavan, and the Supreme Court. New Delhi is a municipality within the NCT, administered by the New Delhi Municipal Council (NDMC), which covers mostly Lutyens' Delhi and a few adjacent areas. The municipal area is [...] New Delhi is a cosmopolitan city due to the multi-ethnic and multi-cultural presence of the vast Indian bureaucracy and political system. The city's capital status has amplified the importance of national events and holidays. National events such as Republic Day, Independence Day and Gandhi Jayanti (Gandhi's birthday) are celebrated with great enthusiasm in New Delhi and the rest of India. On India's Independence Day (15 Augus

In [18]:
results = tavily_tool.invoke("What is the current weather in Bangalore? from weatherapi")
results

[{'url': 'https://timesofindia.indiatimes.com/weather/bengaluru-weather-forecast-today/560001',
  'content': "Today's Weather in Bengaluru: In Bengaluru today, the weather is expected to be Mostly Cloudy with a maximum temperature of 26°C and a minimum of 22°C."},
 {'url': 'https://www.indiatoday.in/weather/bangalore-weather-forecast-today',
  'content': "The minimum temperature in Bangalore today is likely to hover around 21 degrees Celsius, while the maximum temperature might reach 25 degrees Celsius. The mercury level is expected to hover around 21 degrees Celsius throughout the day, with the wind speed around 8.03. The wind will move around 260 degrees with a gust speed of 16.12. The sunrise time is 05:53 AM, while it will set at 06:43 PM on Friday. As per the seven-day weather prediction, the temperature in Bangalore is likely to reach 25 [...] degrees Celsius on Friday, 29 degrees Celsius on Saturday, 30 degrees Celsius on Sunday, 29 degrees Celsius on Monday, 28 degrees Celsius 

In [19]:
results[0]['content']

"Today's Weather in Bengaluru: In Bengaluru today, the weather is expected to be Mostly Cloudy with a maximum temperature of 26°C and a minimum of 22°C."

In [20]:
print(results[0]['content'])

Today's Weather in Bengaluru: In Bengaluru today, the weather is expected to be Mostly Cloudy with a maximum temperature of 26°C and a minimum of 22°C.


In [24]:
results = tavily_tool.invoke("Get Current weather in Zurich, from weatherapi")
results

[{'url': 'https://www.timeanddate.com/weather/switzerland/zurich',
  'content': 'Weather in Zürich, Zurich, Switzerland\n\nSunny.\n\nFeels Like: 59\xa0°FForecast: 85 / 53\xa0°FWind: 1 mph ↑ from North\n\nLocation: | Zurich-Kloten\nCurrent Time: | May 30, 2025 at 9:06:50 am\nLatest Report: | May 30, 2025 at 8:20 am\nVisibility: | 10\xa0mi\nPressure: | 30.27 "Hg\nHumidity: | 82%\nDew Point: | 54\xa0°F\nUpcoming 5 hours\n\nNow | 10:00 am | 11:00 am | 12:00 pm | 1:00 pm | 2:00 pm\n |  |  |  |  | \n59\xa0°F | 65\xa0°F | 69\xa0°F | 72\xa0°F | 75\xa0°F | 77\xa0°F\nSee more hour-by-hour weather\n\nForecast for the next 48 hours [...] Humidity | 73% | 45% | 67% | 93% | 72% | 42% | 65%\nDew Point | 52\xa0°F | 56\xa0°F | 59\xa0°F | 53\xa0°F | 55\xa0°F | 58\xa0°F | 61\xa0°F\nVisibility | 8\xa0mi | 9\xa0mi | 7\xa0mi | 7\xa0mi | 8\xa0mi | 9\xa0mi | 6\xa0mi\nProbability of Precipitation | 0% | 0% | 0% | 0% | 0% | 0% | 0%\n* Updated Friday, May 30, 2025 3:42:58 am Zürich time - Weather by CustomWeathe

In [26]:
print(results[0]['content'])

Weather in Zürich, Zurich, Switzerland

Sunny.

Feels Like: 59 °FForecast: 85 / 53 °FWind: 1 mph ↑ from North

Location: | Zurich-Kloten
Current Time: | May 30, 2025 at 9:06:50 am
Latest Report: | May 30, 2025 at 8:20 am
Visibility: | 10 mi
Pressure: | 30.27 "Hg
Humidity: | 82%
Dew Point: | 54 °F
Upcoming 5 hours

Now | 10:00 am | 11:00 am | 12:00 pm | 1:00 pm | 2:00 pm
 |  |  |  |  | 
59 °F | 65 °F | 69 °F | 72 °F | 75 °F | 77 °F
See more hour-by-hour weather

Forecast for the next 48 hours [...] Humidity | 73% | 45% | 67% | 93% | 72% | 42% | 65%
Dew Point | 52 °F | 56 °F | 59 °F | 53 °F | 55 °F | 58 °F | 61 °F
Visibility | 8 mi | 9 mi | 7 mi | 7 mi | 8 mi | 9 mi | 6 mi
Probability of Precipitation | 0% | 0% | 0% | 0% | 0% | 0% | 0%
* Updated Friday, May 30, 2025 3:42:58 am Zürich time - Weather by CustomWeather, © 2025
14 day forecast, day-by-dayHour-by-hour forecast for next week [...] 74 / 60 °F

Detailed forecast for 14 days

Need some help?

Love Our Site? Become a Supporter

© T

## Exploring the DuckDuckGo Search Tool

Enables you to search for words, documents, images, videos, news, maps and text translation using the DuckDuckGo.com search engine.

In [27]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

# we choose wt-wt to just do generic and not region specific search
# check https://github.com/deedy5/duckduckgo_search?tab=readme-ov-file#regions for more details
wrapper = DuckDuckGoSearchAPIWrapper(region="wt-wt", max_results=2)
ddgs_tool = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")

In [28]:
results = ddgs_tool.invoke("Tell me about New Delhi")
results

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'snippet: One of the country\'s largest urban agglomerations, Delhi sits astride (but primarily on the west bank of) the Yamuna River, a tributary of the Ganges (Ganga) River, about 100 miles (160 km) south of the Himalayas.The national capital territory embraces Old and New Delhi and the surrounding metropolitan region, as well as adjacent rural areas. To the east the territory is bounded by the state ..., title: Delhi | History, Population, Map, & Facts | Britannica, link: https://www.britannica.com/place/Delhi, snippet: Delhi is famous for its history and historic monuments. From prehistoric to its time as a British colony, Delhi has a remarkable history. Delhi is the site of 8 historic cities, each city conquered and re-built by successive rulers. The first city of Delhi was called Lal Kot and it was settled in 1052., title: What is Delhi Famous For? 10 Things that Make Delhi Famous, link: https://suzannewandersdelhi.com/what-is-delhi-famous-for/, snippet: 2. It Has Been the Capita

In [29]:
results = [doc.strip('[]') for doc in results.split('], [')]
results

['snippet: One of the country\'s largest urban agglomerations, Delhi sits astride (but primarily on the west bank of) the Yamuna River, a tributary of the Ganges (Ganga) River, about 100 miles (160 km) south of the Himalayas.The national capital territory embraces Old and New Delhi and the surrounding metropolitan region, as well as adjacent rural areas. To the east the territory is bounded by the state ..., title: Delhi | History, Population, Map, & Facts | Britannica, link: https://www.britannica.com/place/Delhi, snippet: Delhi is famous for its history and historic monuments. From prehistoric to its time as a British colony, Delhi has a remarkable history. Delhi is the site of 8 historic cities, each city conquered and re-built by successive rulers. The first city of Delhi was called Lal Kot and it was settled in 1052., title: What is Delhi Famous For? 10 Things that Make Delhi Famous, link: https://suzannewandersdelhi.com/what-is-delhi-famous-for/, snippet: 2. It Has Been the Capit

In [30]:
results = ddgs_tool.invoke("What is the current weather in Bangalore")
results

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'snippet: The temperature in Bangalore today in the early morning is 20 ° C. If you take into account factors such as wind, humidity and other weather conditions, the temperatures can feel like 20 ° C. The chance of rain in Bangalore in the morning is 0%, and the wind will blow at 27 km/h., title: Bangalore local weather (live): today, hourly weather, link: https://www.weather25.com/asia/india/karnataka/bangalore?page=today, snippet: The current temperature is near the lowest expected 21°C for today. Weather radar - Bengaluru, India Doppler radar displays precipitation for the past 2h and forecasts the next 30min, title: Weather today - Bengaluru, India, link: https://www.weather-atlas.com/en/india/bengaluru, snippet: Bengaluru Rain, Weather: City on Rain Alert Today; Traffic Jams, Power Cuts, and Waterlogging Expected. Bangalore Rains Live Updates: Following the havoc caused by rains in Bengaluru, Karnataka Chief Minister Siddaramaiah on Wednesday said that officials have been given c

In [31]:
results = [doc.strip('[]') for doc in results.split('], [')]
results

['snippet: The temperature in Bangalore today in the early morning is 20 ° C. If you take into account factors such as wind, humidity and other weather conditions, the temperatures can feel like 20 ° C. The chance of rain in Bangalore in the morning is 0%, and the wind will blow at 27 km/h., title: Bangalore local weather (live): today, hourly weather, link: https://www.weather25.com/asia/india/karnataka/bangalore?page=today, snippet: The current temperature is near the lowest expected 21°C for today. Weather radar - Bengaluru, India Doppler radar displays precipitation for the past 2h and forecasts the next 30min, title: Weather today - Bengaluru, India, link: https://www.weather-atlas.com/en/india/bengaluru, snippet: Bengaluru Rain, Weather: City on Rain Alert Today; Traffic Jams, Power Cuts, and Waterlogging Expected. Bangalore Rains Live Updates: Following the havoc caused by rains in Bengaluru, Karnataka Chief Minister Siddaramaiah on Wednesday said that officials have been given 

## Building tools in LangChain

Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

We will start by building a simple tool which does some basic math

In [32]:
from langchain_core.tools import tool


@tool
def multiply(a, b):
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A'}, 'b': {'title': 'B'}}


In [33]:
type(multiply)

langchain_core.tools.structured.StructuredTool

In [34]:
multiply.invoke({"a": 2, "b": 3})

6

In [35]:
multiply.invoke({"a": 2.1, "b": 3.2})

6.720000000000001

In [36]:
multiply.invoke({"a": 2, "b": 'abc'})

'abcabc'

Let's now build a tool with data type enforcing

In [37]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

class CalculatorInput(BaseModel):
    a: float = Field(description="first number")
    b: float = Field(description="second number")


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# we could also use the @tool decorator from before
calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="use to multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True
    )

# Let's inspect some of the attributes associated with the tool.
print(calculator.name)
print(calculator.description)
print(calculator.args)

Calculator
use to multiply numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'number'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'number'}}


In [38]:
calculator.invoke({"a": 2, "b": 3})

6.0

## Explore LLM tool calling with custom tools

An agent is basically an LLM which has the capability to automatically call relevant functions to perform complex or tool-based tasks based on input human prompts.

Tool calling also popularly known as function calling is the ability to reliably enable such LLMs to call external tools and APIs.

We will create some custom tools here and try to see if the LLM can automatically call the right tools based on input prompts

In [47]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import rich


@tool
def add(a: float, b: float) -> float:
    """Adds a and b."""
    print(a + b)


@tool
def multiply(a: float, b: float) -> float:
    """Multiplies a and b."""
    print(a * b)

@tool
def search_web(query: str) -> list:
    """Search the web for a query."""
    tavily_tool = TavilySearchResults(max_results=2)
    results = tavily_tool.invoke(query)
    for result in results:
      print(result['content'])

@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather."""
    if 'weather' not in query: # guides the search tool to use weatherapi
      query = query + " weather from weatherapi"
    tavily_tool = TavilySearchResults(max_results=1)
    result = tavily_tool.invoke(query)
    print(result[0]['content'])


tools = [add, multiply, search_web, get_weather]

## Tool calling for LLMs with native support for tool or function calling

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. While the name implies that the model is performing some action, this is actually not the case! The model is coming up with the arguments to a tool, and actually running the tool (or not) is up to the user or agent defined by the user.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools.



In [41]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

In [42]:
chatgpt_with_tools = chatgpt.bind_tools(tools)

In [43]:
# LLMs are still not perfect in tool calling so you might need to play around with the following prompt
prompt = """
            Given only the tools at your disposal, mention tool calls for the following tasks:
            Do not change the query given for any search tasks
            1. What is 2.1 times 3.5
            2. What is 11 + 29
            3. What is the current weather in Greenland today
            4. Can you tell me about Greenland and its capital
         """

results = chatgpt_with_tools.invoke(prompt)

In [44]:
results

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oTTvcTahnf8VLjozq9Gb5yGk', 'function': {'arguments': '{"a": 2.1, "b": 3.5}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_1IjllTbd9RWTkZASypmEY7F3', 'function': {'arguments': '{"a": 11, "b": 29}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_2QtuLmAstVM82XlkvnybrJfx', 'function': {'arguments': '{"query": "Greenland"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_e7Hv6mn6b71zjMC6qWYc4OyS', 'function': {'arguments': '{"query": "Greenland and its capital"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 199, 'total_tokens': 286, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad

In [45]:
results.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.1, 'b': 3.5},
  'id': 'call_oTTvcTahnf8VLjozq9Gb5yGk',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 29},
  'id': 'call_1IjllTbd9RWTkZASypmEY7F3',
  'type': 'tool_call'},
 {'name': 'get_weather',
  'args': {'query': 'Greenland'},
  'id': 'call_2QtuLmAstVM82XlkvnybrJfx',
  'type': 'tool_call'},
 {'name': 'search_web',
  'args': {'query': 'Greenland and its capital'},
  'id': 'call_e7Hv6mn6b71zjMC6qWYc4OyS',
  'type': 'tool_call'}]

In [48]:
toolkit = {
    "add": add,
    "multiply": multiply,
    "search_web": search_web,
    "get_weather": get_weather
}

for tool_call in results.tool_calls:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["args"])
    print()

Calling tool: multiply
7.3500000000000005

Calling tool: add
40.0

Calling tool: get_weather
Are you planning a holiday with hopefully nice weather in Greenland in May 2025? ... 1.4 mm | 0.1 inch. 30. May, 2 °C | 36 °F, 5 °C | 41 °F, -0 °C | 31 °F, 1 °C |

Calling tool: search_web
Nuuk (Greenlandic pronunciation: [nuːk] ⓘ; Danish: Nuuk,[1] formerly Godthåb [ˈkʌtˌhɔˀp])[2] is the capital and most populous city of Greenland, an autonomous territory in the Kingdom of Denmark. Nuuk is the seat of government and the territory's largest cultural and economic center. It is also the seat of government for the Sermersooq municipality. In January 2025, it had a population of 20,113[3] – more than a third of the territory's population – making it one of the smallest capital cities [...] Government

As the capital of Greenland, Nuuk is the administrative center of the country, containing all of the important government buildings and institutions. The public sector bodies are also the town's larges

In [49]:
tools

[StructuredTool(name='add', description='Adds a and b.', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7a3b82789760>),
 StructuredTool(name='multiply', description='Multiplies a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x7a3b82afeac0>),
 StructuredTool(name='search_web', description='Search the web for a query.', args_schema=<class 'langchain_core.utils.pydantic.search_web'>, func=<function search_web at 0x7a3b8397b740>),
 StructuredTool(name='get_weather', description='Search weatherapi to get the current weather.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x7a3bcb0360c0>)]

### Tool calling for LLMs without native support for tool or function calling

Some models like ChatGPT have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling.

Here we will explore an alternative method to invoke tools if you're using a model that does not natively support tool calling (even though we use ChatGPT here which supports it, we will assume it could be any LLM which doesn't support tool calling).

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools.

In [50]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

add(a: float, b: float) -> float - Adds a and b.
multiply(a: float, b: float) -> float - Multiplies a and b.
search_web(query: str) -> list - Search the web for a query.
get_weather(query: str) -> list - Search weatherapi to get the current weather.


In [51]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user instructions, for each instruction do the following:
 - Return the name and input of the tool to use.
 - Return your response as a JSON blob with 'name' and 'arguments' keys.
 - The `arguments` should be a dictionary, with keys corresponding
   to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)

In [52]:
instructions = [
                  {"input" : "What is 2.1 times 3.5"},
                  {"input" : "What is 11 + 29"},
                  {"input" : "What is the current weather in Greenland"},
                  {"input" : "Tell me about Greenland and its capital" }
               ]

In [53]:
from langchain_core.output_parsers import JsonOutputParser

chain = (prompt
            |
         chatgpt
            |
         JsonOutputParser()
        )

In [54]:
responses = chain.map().invoke(instructions)

In [55]:
responses

[{'name': 'multiply', 'arguments': {'a': 2.1, 'b': 3.5}},
 {'name': 'add', 'arguments': {'a': 11, 'b': 29}},
 {'name': 'get_weather', 'arguments': {'query': 'Greenland'}},
 {'name': 'search_web', 'arguments': {'query': 'Greenland and its capital'}}]

In [56]:
toolkit = {
    "add": add,
    "multiply": multiply,
    "search_web": search_web,
    "get_weather": get_weather
}
# sometimes get_weather may not get the right results from weatherapi
# so just run it a few times in case you get an error, later we will use a direct weather api
for tool_call in responses:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["arguments"])
    print()

Calling tool: multiply
7.3500000000000005

Calling tool: add
40.0

Calling tool: get_weather
Are you planning a holiday with hopefully nice weather in Greenland in May 2025? ... 1.4 mm | 0.1 inch. 30. May, 2 °C | 36 °F, 5 °C | 41 °F, -0 °C | 31 °F, 1 °C |

Calling tool: search_web
Nuuk (Greenlandic pronunciation: [nuːk] ⓘ; Danish: Nuuk,[1] formerly Godthåb [ˈkʌtˌhɔˀp])[2] is the capital and most populous city of Greenland, an autonomous territory in the Kingdom of Denmark. Nuuk is the seat of government and the territory's largest cultural and economic center. It is also the seat of government for the Sermersooq municipality. In January 2025, it had a population of 20,113[3] – more than a third of the territory's population – making it one of the smallest capital cities [...] Government

As the capital of Greenland, Nuuk is the administrative center of the country, containing all of the important government buildings and institutions. The public sector bodies are also the town's larges